In [1]:
import numpy as np
import pandas as pd
import vectorbt as vbt
import pandas_ta as ta
import warnings
from datetime import datetime
import time
import plotly.figure_factory as ff
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

In [2]:
symbols = ['USDJPY=X']
start_date = '1996-10-30'
end_date = '2001-10-30'
price = vbt.YFData.download(symbols,start=start_date,end=end_date,internval='1D',missing_index='drop')


In [3]:
df = pd.DataFrame()
df = price.data[symbols[0]]
df.drop(['Volume','Dividends','Stock Splits'],axis=1,inplace=True)

In [4]:
df

,Open,High,Low,Close
Date,,,,
1996-10-30 00:00:00+00:00,114.370003,114.480003,113.610001,114.180000
1996-11-01 00:00:00+00:00,113.500000,113.500000,113.500000,113.500000
1996-11-04 00:00:00+00:00,113.279999,113.980003,112.949997,113.879997
1996-11-05 00:00:00+00:00,113.709999,114.330002,113.449997,114.250000
1996-11-06 00:00:00+00:00,114.230003,114.680000,113.650002,113.949997
...,...,...,...,...
2001-10-22 23:00:00+00:00,122.339996,123.019997,122.199997,122.510002
2001-10-23 23:00:00+00:00,122.540001,122.959999,122.279999,122.809998
2001-10-24 23:00:00+00:00,122.800003,123.349998,122.410004,122.879997


In [5]:
df.index = pd.to_datetime(df.index).strftime("%Y-%m-%d")

In [6]:
df.index=pd.to_datetime(df.index)

In [7]:
df

,Open,High,Low,Close
Date,,,,
1996-10-30,114.370003,114.480003,113.610001,114.180000
1996-11-01,113.500000,113.500000,113.500000,113.500000
1996-11-04,113.279999,113.980003,112.949997,113.879997
1996-11-05,113.709999,114.330002,113.449997,114.250000
1996-11-06,114.230003,114.680000,113.650002,113.949997
...,...,...,...,...
2001-10-22,122.339996,123.019997,122.199997,122.510002
2001-10-23,122.540001,122.959999,122.279999,122.809998
2001-10-24,122.800003,123.349998,122.410004,122.879997


In [8]:
bb = ta.bbands(df.Close,length=14,std=2,mamode="ema")

In [10]:
bb

,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0
Date,,,,,
1996-10-30,NaN,NaN,NaN,NaN,NaN
1996-11-01,NaN,NaN,NaN,NaN,NaN
1996-11-04,NaN,NaN,NaN,NaN,NaN
1996-11-05,NaN,NaN,NaN,NaN,NaN
1996-11-06,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2001-10-22,119.570523,121.081125,122.591727,2.495190,0.972950
2001-10-23,119.578444,121.311641,123.044839,2.857430,0.932252
2001-10-24,119.633374,121.520755,123.408136,3.106269,0.860087


In [11]:
Entry = df.Close < bb['BBL_14_2.0']
Exit = df.Close > bb['BBU_14_2.0']

In [12]:
Entry,Exit

(Date
 1996-10-30    False
 1996-11-01    False
 1996-11-04    False
 1996-11-05    False
 1996-11-06    False
               ...  
 2001-10-22    False
 2001-10-23    False
 2001-10-24    False
 2001-10-25    False
 2001-10-29    False
 Length: 1290, dtype: bool,
 Date
 1996-10-30    False
 1996-11-01    False
 1996-11-04    False
 1996-11-05    False
 1996-11-06    False
               ...  
 2001-10-22    False
 2001-10-23    False
 2001-10-24    False
 2001-10-25    False
 2001-10-29    False
 Length: 1290, dtype: bool)

In [13]:
Entry.shift(1)
Exit.shift(1)
Entry.fillna(False,inplace =True)
Exit.fillna(False,inplace=True)
Entry_signal_vectorbt = ta.tsignals(Entry,asbool=True,append=True)
Exit_signal_vectorbt = ta.tsignals(Exit,asbool=True,append=True)

In [26]:
port=vbt.Portfolio.from_signals(df.Close,
                               entries=Entry_signal_vectorbt.TS_Entries,
                               exits =Exit_signal_vectorbt.TS_Entries,
                               freq='D',
                               init_cash=10000,
                               fees=0.0025,
                               slippage=0.0025,
                               size = 10,
                               sl_stop=0.05, sl_trail=True, 
                               direction=0)

In [27]:
port.plot().show()